In [1]:
import numpy as np

In [2]:
import tensorflow as tf

In [3]:
from tensorflow import keras

In [4]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [5]:
X_train_full = X_train_full[:30000]
y_train_full = y_train_full[:30000]

In [6]:
X_test = X_test[:5000]
y_test = y_test[:5000]

In [7]:
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

In [8]:
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]

In [9]:
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [14]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A), (X[y_5_or_6], y_B))

In [15]:
(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)

In [27]:
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)

In [28]:
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)

In [55]:
keras.backend.clear_session()

In [56]:
tf.random.set_seed(42)
np.random.seed(42)

In [57]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [58]:
optimizer = keras.optimizers.SGD(learning_rate=1e-3)

In [59]:
model_A.compile(loss= "sparse_categorical_crossentropy",
    optimizer= optimizer,
    metrics=["accuracy"])

In [60]:
history = model_A.fit(X_train_A, y_train_A, epochs=5,
            validation_data=(X_valid_A, y_valid_A))

Epoch 1/5
622/622 [==============================] - 3s 4ms/step - loss: 0.7979 - accuracy: 0.7498 - val_loss: 0.4762 - val_accuracy: 0.8490
Epoch 2/5
622/622 [==============================] - 2s 3ms/step - loss: 0.4260 - accuracy: 0.8582 - val_loss: 0.3921 - val_accuracy: 0.8652
Epoch 3/5
622/622 [==============================] - 2s 3ms/step - loss: 0.3702 - accuracy: 0.8737 - val_loss: 0.3700 - val_accuracy: 0.8712
Epoch 4/5
622/622 [==============================] - 2s 3ms/step - loss: 0.3418 - accuracy: 0.8812 - val_loss: 0.3285 - val_accuracy: 0.8869
Epoch 5/5
622/622 [==============================] - 2s 3ms/step - loss: 0.3240 - accuracy: 0.8869 - val_loss: 0.3229 - val_accuracy: 0.8884


In [61]:
model_A.save("my_model_A.h5")

In [62]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="softmax"))

In [64]:
model_B.compile(loss= "binary_crossentropy",
    optimizer= optimizer,
    metrics=["accuracy"])

In [65]:
history = model_B.fit(X_train_B, y_train_B, epochs=5,
            validation_data=(X_valid_B, y_valid_B))

Epoch 1/5
161/161 [==============================] - 1s 5ms/step - loss: 0.2678 - accuracy: 0.5050 - val_loss: 0.1301 - val_accuracy: 0.5000
Epoch 2/5
161/161 [==============================] - 1s 4ms/step - loss: 0.0910 - accuracy: 0.5050 - val_loss: 0.0765 - val_accuracy: 0.5000
Epoch 3/5
161/161 [==============================] - 1s 3ms/step - loss: 0.0573 - accuracy: 0.5050 - val_loss: 0.0565 - val_accuracy: 0.5000
Epoch 4/5
161/161 [==============================] - 1s 4ms/step - loss: 0.0423 - accuracy: 0.5050 - val_loss: 0.0456 - val_accuracy: 0.5000
Epoch 5/5
161/161 [==============================] - 1s 4ms/step - loss: 0.0335 - accuracy: 0.5050 - val_loss: 0.0389 - val_accuracy: 0.5000


In [68]:
model_B.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 300)               235500    
_________________________________________________________________
dense_7 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_8 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_9 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_10 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                

In [69]:
model_A_clone = keras.models.clone_model(model_A)

In [70]:
model_A_clone.set_weights(model_A.get_weights())

In [71]:
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])

In [72]:
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [73]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

In [74]:
model_B_on_A.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                

In [75]:
model_B_on_A.compile(loss="binary_crossentropy",
         optimizer=optimizer,
         metrics=["accuracy"])

In [76]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=5,
                   validation_data=(X_valid_B, y_valid_B))

Epoch 1/5
161/161 [==============================] - 1s 4ms/step - loss: 0.3751 - accuracy: 0.8318 - val_loss: 0.2783 - val_accuracy: 0.9138
Epoch 2/5
161/161 [==============================] - 0s 3ms/step - loss: 0.2169 - accuracy: 0.9463 - val_loss: 0.1870 - val_accuracy: 0.9574
Epoch 3/5
161/161 [==============================] - 0s 3ms/step - loss: 0.1544 - accuracy: 0.9742 - val_loss: 0.1448 - val_accuracy: 0.9726
Epoch 4/5
161/161 [==============================] - 0s 3ms/step - loss: 0.1225 - accuracy: 0.9820 - val_loss: 0.1210 - val_accuracy: 0.9838
Epoch 5/5
161/161 [==============================] - 0s 2ms/step - loss: 0.1033 - accuracy: 0.9856 - val_loss: 0.1057 - val_accuracy: 0.9848


In [79]:
model_B.evaluate(X_test_B,y_test_B)

31/31 [==============================] - 0s 2ms/step - loss: 0.0319 - accuracy: 0.4984


[0.03187718987464905, 0.49844881892204285]

In [80]:
model_B_on_A.evaluate(X_test_B,y_test_B)

31/31 [==============================] - 0s 2ms/step - loss: 0.0990 - accuracy: 0.9886


[0.0990130677819252, 0.988624632358551]